In [1]:
import math
import random
from collections import deque, namedtuple

import gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from binance import Client
from gym import spaces

### Данные binance ETHUSDT

In [2]:
# Инициализация клиента Binance (публичный доступ)
client = Client()


# Загрузка исторических данных (свечи)
def get_klines(
    symbol="ETHUSDT",
    interval="1m",
    start_date="6 day ago UTC",
    end_date="2 day ago UTC",
):
    try:
        klines = client.get_historical_klines(
            symbol=symbol, interval=interval, start_str=start_date, end_str=end_date
        )
        columns = [
            "time",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "close_time",
            "quote_volume",
            "trades",
            "taker_buy_base",
            "taker_buy_quote",
            "ignore",
        ]
        df = pd.DataFrame(klines, columns=columns, dtype=float)
        df["time"] = pd.to_datetime(df["time"], unit="ms")
        return df[["time", "open", "high", "low", "close", "volume", "trades"]]
    except Exception as e:
        print(f"Ошибка при получении данных: {e}")
        return None


df = get_klines(symbol="ETHUSDT", interval="1m")
print(df.head())

# Разбиваем данные на обучающую и тестовую выборки
if df is not None:
    N = len(df)
    ratio = 0.8
    df_test = df[int(N * ratio) :].reset_index(drop=True)
    df = df[: int(N * ratio)].reset_index(drop=True)
else:
    df = pd.DataFrame(
        columns=["time", "open", "high", "low", "close", "volume", "trades"]
    )
    df_test = pd.DataFrame(
        columns=["time", "open", "high", "low", "close", "volume", "trades"]
    )

                 time     open     high      low    close    volume  trades
0 2025-03-17 16:10:00  1923.49  1924.36  1922.03  1922.17  376.2700  1728.0
1 2025-03-17 16:11:00  1922.16  1922.95  1920.21  1920.70  173.0118  1689.0
2 2025-03-17 16:12:00  1920.70  1921.72  1919.69  1920.41  265.8190  1691.0
3 2025-03-17 16:13:00  1920.42  1921.84  1920.01  1920.70  418.7001  1556.0
4 2025-03-17 16:14:00  1920.71  1921.64  1920.00  1920.46  271.7455  1210.0


### Среда

In [3]:
class MarketMakingEnv(gym.Env):
    """
    Окружение маркет-мейкинга по Avellaneda-Stoikov с использованием свечных данных.
    Состояние: [inventory, history(relative price changes, длина=16), PnL].
    Действие: мультидискретный выбор (γ, κ, Δσ), где:
      - γ: коэффициент аверсии к риску,
      - κ: интенсивность ликвидности,
      - Δσ: поправка к оценённой волатильности.
    """

    def __init__(
        self, df, gamma_values, kappa_values, vol_corr_values, T=60.0, history_len=16
    ):
        super(MarketMakingEnv, self).__init__()
        self.prices = df.close.values
        self.prices_high = df.high.values
        self.prices_low = df.low.values
        self.N = len(df)
        self.gamma_values = gamma_values
        self.kappa_values = kappa_values
        self.vol_corr_values = vol_corr_values
        self.T = T  # торговый горизонт (сек) для расчёта оптимальных котировок
        self.history_len = history_len
        # Пространство действий: мультидискретное – [#γ, #κ, #Δσ]
        self.action_space = spaces.MultiDiscrete(
            [len(gamma_values), len(kappa_values), len(vol_corr_values)]
        )
        # Пространство состояний: размер = 1 (inventory) + history_len (относительные изменения цены) + 1 (PnL)
        low = np.concatenate(
            (np.array([-100.0]), np.full((history_len,), -1.0), np.array([-1e9]))
        )
        high = np.concatenate(
            (np.array([100.0]), np.full((history_len,), 1.0), np.array([1e9]))
        )
        self.observation_space = spaces.Box(low, high, dtype=np.float32)
        self.initial_cash = 10000.0
        self.fee_rate = 0.0  # комиссия

    def reset(self):
        self.t = 0
        self.inventory = 0.0
        self.cash = self.initial_cash
        # Инициализируем историю относительных изменений цены нулями
        self.price_change_history = [0.0] * self.history_len
        # Начальное наблюдение: [inventory, history, pnl]
        pnl = self.cash - self.initial_cash
        state = np.concatenate(
            ([self.inventory], np.array(self.price_change_history), [pnl])
        )
        return state

    def step(self, action):
        # Раскодирование мультидискретного действия:
        n_gamma = len(self.gamma_values)
        n_kappa = len(self.kappa_values)
        n_vol = len(self.vol_corr_values)
        if isinstance(action, (np.ndarray, list, tuple)):
            gamma_idx = int(action[0])
            kappa_idx = int(action[1])
            vol_corr_idx = int(action[2])
        else:
            action = int(action)
            gamma_idx = action // (n_gamma * n_vol)
            rem = action % (n_kappa * n_vol)
            kappa_idx = rem // n_vol
            vol_corr_idx = rem % n_vol

        gamma = self.gamma_values[gamma_idx]
        kappa = self.kappa_values[kappa_idx]
        vol_corr = self.vol_corr_values[vol_corr_idx]

        mid_price = self.prices[self.t]
        # Оценка волатильности на окне длиной T секунд
        sigma_est = 0.0
        if self.t > 1:
            start = max(0, self.t - int(self.T))
            window_prices = self.prices[start : self.t + 1]
            if len(window_prices) > 1:
                returns = np.diff(window_prices) / window_prices[:-1]
                sigma_est = np.std(returns)
        effective_sigma = sigma_est * vol_corr
        if effective_sigma < 1e-8:
            effective_sigma = 1e-8

        # Расчет цены резервирования и оптимального спреда
        reservation_price = (
            mid_price - self.inventory * gamma * (effective_sigma**2) * self.T
        )
        delta = gamma * (effective_sigma**2) * self.T + (1.0 / gamma) * math.log(
            1 + gamma / kappa
        )
        bid_price = reservation_price - delta / 2
        ask_price = reservation_price + delta / 2

        done = False
        reward = 0.0
        if self.t < self.N - 1:
            next_price = self.prices[self.t + 1]
            if self.prices_high[self.t + 1] >= ask_price:
                self.inventory -= 1.0
                self.cash += ask_price * (1.0 - self.fee_rate)
            if self.prices_low[self.t + 1] <= bid_price:
                self.inventory += 1.0
                self.cash -= bid_price * (1.0 + self.fee_rate)
            current_value = self.cash + self.inventory * next_price
            prev_value = self.cash + self.inventory * mid_price
            pnl_change = current_value - prev_value
            reward = pnl_change - 0.001 * abs(self.inventory)
            self.t += 1
        else:
            done = True
            current_value = self.cash + self.inventory * mid_price
            reward = current_value - self.initial_cash

        # Обновляем историю относительных изменений цены:
        if self.t > 0:
            # Рассчитываем относительное изменение цены между предыдущей и текущей свечами
            rel_change = (self.prices[self.t] - self.prices[self.t - 1]) / self.prices[
                self.t - 1
            ]
        else:
            rel_change = 0.0
        # Сдвигаем историю (удаляем самый старый элемент и добавляем новый)
        self.price_change_history.pop(0)
        self.price_change_history.append(rel_change)

        if not done:
            new_mid = self.prices[self.t]
            pnl = self.cash + self.inventory * new_mid - self.initial_cash
            # Собираем наблюдение: [inventory, history(16 значений), pnl]
            obs = np.concatenate(
                ([self.inventory], np.array(self.price_change_history), [pnl])
            ).astype(np.float32)
        else:
            obs = None
        return obs, reward, done, {}


### Агенты

In [4]:
###############################################################################
# DQN-агент
###############################################################################
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


Transition = namedtuple(
    "Transition", ["state", "action", "reward", "next_state", "done"]
)


class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, *args):
        self.buffer.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)


class DQNAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        buffer_capacity=10000,
        batch_size=64,
        target_update=1000,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.q_network = QNetwork(state_dim, action_dim).to(self.device)
        self.target_network = QNetwork(state_dim, action_dim).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.gamma = gamma
        self.buffer = ReplayBuffer(buffer_capacity)
        self.batch_size = batch_size
        self.target_update = target_update
        self.steps_done = 0
        self.action_dim = action_dim

    def select_action(self, state, epsilon):
        if random.random() < epsilon:
            return random.randrange(self.action_dim)
        else:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.q_network(state_tensor)
            return q_values.argmax().item()

    def push_transition(self, state, action, reward, next_state, done):
        self.buffer.push(state, action, reward, next_state, done)

    def update(self):
        if len(self.buffer) < self.batch_size:
            return
        transitions = self.buffer.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        state_batch = torch.FloatTensor(batch.state).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).unsqueeze(1).to(self.device)
        non_final_mask = torch.tensor(
            [s is not None for s in batch.next_state],
            dtype=torch.bool,
            device=self.device,
        )
        non_final_next_states = torch.FloatTensor(
            [s for s in batch.next_state if s is not None]
        ).to(self.device)
        done_batch = torch.FloatTensor(batch.done).unsqueeze(1).to(self.device)
        q_values = self.q_network(state_batch).gather(1, action_batch)
        next_q_values = torch.zeros(self.batch_size, 1).to(self.device)
        if non_final_next_states.size(0) > 0:
            next_q_values[non_final_mask] = self.target_network(
                non_final_next_states
            ).max(1, keepdim=True)[0]
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values
        loss = nn.MSELoss()(q_values, expected_q_values.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.steps_done += 1
        if self.steps_done % self.target_update == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())


###############################################################################
# A2C-агент
###############################################################################
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.actor = nn.Linear(64, action_dim)
        self.critic = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        logits = self.actor(x)
        value = self.critic(x)
        return logits, value


class A2CAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        value_coef=0.5,
        entropy_coef=0.01,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef

    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        logits, value = self.model(state_tensor)
        probs = torch.softmax(logits, dim=1)
        action_tensor = torch.multinomial(probs, num_samples=1)
        log_prob = torch.log(probs.gather(1, action_tensor) + 1e-10)
        return (
            action_tensor.item(),
            log_prob.squeeze(),
            value.squeeze(),
            logits.squeeze(),
        )

    def update(self, trajectories):
        log_probs = torch.stack([t[2] for t in trajectories]).to(self.device)
        values = torch.stack([t[3] for t in trajectories]).to(self.device)
        rewards = [t[4] for t in trajectories]
        dones = [t[5] for t in trajectories]
        logits_list = torch.stack([t[6] for t in trajectories]).to(self.device)
        R = 0
        returns = []
        for reward, done in zip(reversed(rewards), reversed(dones)):
            R = reward + self.gamma * R * (1 - done)
            returns.insert(0, R)
        returns = torch.FloatTensor(returns).to(self.device)
        advantages = returns - values
        actor_loss = -(log_probs * advantages.detach()).mean()
        critic_loss = advantages.pow(2).mean()
        probs = torch.softmax(logits_list, dim=1)
        entropy = -(probs * torch.log(probs + 1e-10)).sum(dim=1).mean()
        loss = actor_loss + self.value_coef * critic_loss - self.entropy_coef * entropy
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


###############################################################################
# PPO-агент
###############################################################################
class PPOAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        clip_coef=0.2,
        value_coef=0.5,
        entropy_coef=0.01,
        ppo_epochs=4,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma
        self.clip_coef = clip_coef
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        self.ppo_epochs = ppo_epochs

    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        logits, value = self.model(state_tensor)
        probs = torch.softmax(logits, dim=1)
        action_tensor = torch.multinomial(probs, num_samples=1)
        log_prob = torch.log(probs.gather(1, action_tensor) + 1e-10)
        return action_tensor.item(), log_prob.squeeze(), value.squeeze()

    def update(self, trajectories):
        states = torch.FloatTensor([t[0] for t in trajectories]).to(self.device)
        actions = (
            torch.LongTensor([t[1] for t in trajectories]).unsqueeze(1).to(self.device)
        )
        old_log_probs = torch.stack([t[2] for t in trajectories]).to(self.device)
        old_values = torch.stack([t[3] for t in trajectories]).to(self.device)
        rewards = [t[4] for t in trajectories]
        dones = [t[5] for t in trajectories]
        R = 0
        returns = []
        for reward, done in zip(reversed(rewards), reversed(dones)):
            R = reward + self.gamma * R * (1 - done)
            returns.insert(0, R)
        returns = torch.FloatTensor(returns).to(self.device)
        old_log_probs = old_log_probs.detach()
        old_values = old_values.detach()
        advantages = returns - old_values
        for _ in range(self.ppo_epochs):
            logits, values = self.model(states)
            values = values.squeeze(1)
            probs = torch.softmax(logits, dim=1)
            new_log_probs = torch.log(probs.gather(1, actions) + 1e-10).squeeze(1)
            ratio = torch.exp(new_log_probs - old_log_probs)
            adv_detached = advantages.detach()
            surr1 = ratio * adv_detached
            surr2 = (
                torch.clamp(ratio, 1.0 - self.clip_coef, 1.0 + self.clip_coef)
                * adv_detached
            )
            actor_loss = -torch.min(surr1, surr2).mean()
            critic_loss = (returns - values).pow(2).mean()
            entropy = -(probs * torch.log(probs + 1e-10)).sum(dim=1).mean()
            loss = (
                actor_loss + self.value_coef * critic_loss - self.entropy_coef * entropy
            )
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


In [5]:
###############################################################################
# Функции для вычисления метрик: максимальная просадка и коэффициент Шарпа
###############################################################################
def compute_max_drawdown(pnl_history):
    pnl_array = np.array(pnl_history)
    running_max = np.maximum.accumulate(pnl_array)
    drawdowns = pnl_array - running_max
    max_drawdown = drawdowns.min()
    return max_drawdown


def compute_sharpe_ratio(pnl_history):
    pnl_array = np.array(pnl_history)
    returns = np.diff(pnl_array)
    if returns.std() == 0:
        return 0.0
    sharpe = returns.mean() / returns.std() * np.sqrt(len(returns))
    return sharpe


###############################################################################
# Определяем дискретные наборы параметров для оптимизации
###############################################################################
gamma_values = [0.005, 0.01, 0.05, 0.1, 0.5, 1.0]
kappa_values = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]
vol_corr_values = [0.5, -0.75, 1.0, 0.25, 0.5]  # поправка к sigma

# Создаем окружение с новым мультидискретным пространством действий
env = MarketMakingEnv(
    df=df,
    gamma_values=gamma_values,
    kappa_values=kappa_values,
    vol_corr_values=vol_corr_values,
    T=60.0,
    history_len=16,
)
state_dim = env.observation_space.shape[0]
if isinstance(env.action_space, spaces.MultiDiscrete):
    action_dim = int(np.prod(env.action_space.nvec))
else:
    action_dim = env.action_space.n


/home/strike/work/penv/deep/lib/python3.13/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


### Обучение

In [12]:
###############################################################################
# Инициализируем агентов DQN, A2C и PPO
###############################################################################

dqn_agent = DQNAgent(
    state_dim,
    action_dim,
    lr=1e-3,
    gamma=0.99,
    buffer_capacity=10000,
    batch_size=64,
    target_update=1000,
)
a2c_agent = A2CAgent(
    state_dim, action_dim, lr=1e-3, gamma=0.99, value_coef=0.5, entropy_coef=0.01
)
ppo_agent = PPOAgent(
    state_dim,
    action_dim,
    lr=1e-3,
    gamma=0.99,
    clip_coef=0.2,
    value_coef=0.5,
    entropy_coef=0.01,
    ppo_epochs=4,
)

###############################################################################
# Параметры обучения
###############################################################################
num_episodes = 20  # для DQN
num_episodes_a2c = 20  # для A2C
num_episodes_ppo = 20  # для PPO
epsilon_start = 1.0
epsilon_final = 0.1
epsilon_decay = 300

###############################################################################
# Обучение DQN-агента
###############################################################################
print("Обучение DQN-агента...")
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0.0
    done = False
    while not done:
        epsilon = epsilon_final + (epsilon_start - epsilon_final) * math.exp(
            -env.t / epsilon_decay
        )
        action = dqn_agent.select_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        dqn_agent.push_transition(state, action, reward, next_state, done)
        dqn_agent.update()
        state = next_state if next_state is not None else state
        total_reward += reward
    print(
        f"DQN Эпизод {episode+1}/{num_episodes}: суммарное вознаграждение = {total_reward:.2f}"
    )

###############################################################################
# Обучение A2C-агента
###############################################################################
print("\nОбучение A2C-агента...")
rollout_length = 10
for episode in range(num_episodes_a2c):
    state = env.reset()
    trajectories = []
    total_reward = 0.0
    done = False
    while not done:
        for _ in range(rollout_length):
            action, log_prob, value, logits = a2c_agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectories.append(
                (state, action, log_prob, value, reward, float(done), logits)
            )
            total_reward += reward
            if done:
                break
            state = next_state
        a2c_agent.update(trajectories)
        trajectories = []
    print(
        f"A2C Эпизод {episode+1}/{num_episodes_a2c}: суммарное вознаграждение = {total_reward:.2f}"
    )

###############################################################################
# Обучение PPO-агента
###############################################################################
print("\nОбучение PPO-агента...")
rollout_length_ppo = 20
for episode in range(num_episodes_ppo):
    state = env.reset()
    trajectories = []
    total_reward = 0.0
    done = False
    while not done:
        for _ in range(rollout_length_ppo):
            action, log_prob, value = ppo_agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectories.append((state, action, log_prob, value, reward, float(done)))
            total_reward += reward
            if done:
                break
            state = next_state
        ppo_agent.update(trajectories)
        trajectories = []
    print(
        f"PPO Эпизод {episode+1}/{num_episodes_ppo}: суммарное вознаграждение = {total_reward:.2f}"
    )


Обучение DQN-агента...
DQN Эпизод 1/20: суммарное вознаграждение = 180.33
DQN Эпизод 2/20: суммарное вознаграждение = -40.50
DQN Эпизод 3/20: суммарное вознаграждение = 160.91
DQN Эпизод 4/20: суммарное вознаграждение = 363.26
DQN Эпизод 5/20: суммарное вознаграждение = 80.09
DQN Эпизод 6/20: суммарное вознаграждение = -56.10
DQN Эпизод 7/20: суммарное вознаграждение = 38.27
DQN Эпизод 8/20: суммарное вознаграждение = -198.75
DQN Эпизод 9/20: суммарное вознаграждение = 355.97
DQN Эпизод 10/20: суммарное вознаграждение = 1046.52
DQN Эпизод 11/20: суммарное вознаграждение = 466.70
DQN Эпизод 12/20: суммарное вознаграждение = 437.95
DQN Эпизод 13/20: суммарное вознаграждение = 995.75
DQN Эпизод 14/20: суммарное вознаграждение = -150.07
DQN Эпизод 15/20: суммарное вознаграждение = 493.17
DQN Эпизод 16/20: суммарное вознаграждение = 3.27
DQN Эпизод 17/20: суммарное вознаграждение = 883.49
DQN Эпизод 18/20: суммарное вознаграждение = -413.00
DQN Эпизод 19/20: суммарное вознаграждение = -226.

### Тесты

In [13]:
def run_agent(env, agent, agent_type="dqn"):
    state = env.reset()
    pnl_history = []
    done = False
    while not done:
        if agent_type == "dqn":
            action = agent.select_action(state, epsilon=0.0)
        elif agent_type == "a2c":
            action, _, _, _ = agent.select_action(state)
        elif agent_type == "ppo":
            action, _, _ = agent.select_action(state)
        else:
            action = 0
        next_state, _, done, _ = env.step(action)
        if state is not None:
            pnl_history.append(state[-1])  # последний элемент состояния – PnL
        state = next_state if next_state is not None else state
    return pnl_history


###############################################################################
# Тестирование агентов на отложенной выборке и вычисление метрик
###############################################################################
env_test = MarketMakingEnv(
    df=df_test,
    gamma_values=gamma_values,
    kappa_values=kappa_values,
    vol_corr_values=vol_corr_values,
    T=60.0,
    history_len=16,
)
dqn_pnl = run_agent(env_test, dqn_agent, agent_type="dqn")
env_test = MarketMakingEnv(
    df=df_test,
    gamma_values=gamma_values,
    kappa_values=kappa_values,
    vol_corr_values=vol_corr_values,
    T=60.0,
    history_len=16,
)
a2c_pnl = run_agent(env_test, a2c_agent, agent_type="a2c")
env_test = MarketMakingEnv(
    df=df_test,
    gamma_values=gamma_values,
    kappa_values=kappa_values,
    vol_corr_values=vol_corr_values,
    T=60.0,
    history_len=16,
)
ppo_pnl = run_agent(env_test, ppo_agent, agent_type="ppo")

dqn_max_dd = compute_max_drawdown(dqn_pnl)
dqn_sharpe = compute_sharpe_ratio(dqn_pnl)
a2c_max_dd = compute_max_drawdown(a2c_pnl)
a2c_sharpe = compute_sharpe_ratio(a2c_pnl)
ppo_max_dd = compute_max_drawdown(ppo_pnl)
ppo_sharpe = compute_sharpe_ratio(ppo_pnl)

print("\nРезультаты тестирования:")
print(
    f"DQN итоговый PnL: {dqn_pnl[-1]:.2f}, Max Drawdown: {dqn_max_dd:.2f}, Sharpe Ratio: {dqn_sharpe:.2f}"
)
print(
    f"A2C итоговый PnL: {a2c_pnl[-1]:.2f}, Max Drawdown: {a2c_max_dd:.2f}, Sharpe Ratio: {a2c_sharpe:.2f}"
)
print(
    f"PPO итоговый PnL: {ppo_pnl[-1]:.2f}, Max Drawdown: {ppo_max_dd:.2f}, Sharpe Ratio: {ppo_sharpe:.2f}"
)



Результаты тестирования:
DQN итоговый PnL: 0.00, Max Drawdown: 0.00, Sharpe Ratio: 0.00
A2C итоговый PnL: 33.78, Max Drawdown: -19.09, Sharpe Ratio: 0.80
PPO итоговый PnL: 33.78, Max Drawdown: -19.09, Sharpe Ratio: 0.80


Выводы:  
Результаты все еще не внушают оптимизма